In [14]:
import pandas as pd
from scipy.special import logit

In [15]:
label_df = pd.read_csv('../processed_data/1976-2020-president-two-party.csv')

inc_df = pd.read_csv('../processed_data/incumbency.csv').set_index('year')

label_df = pd.merge(label_df, inc_df, on='year')
label_df['result'] = label_df['rep_pct']
label_df['inc_pct'] = label_df['rep_incumbent'] * label_df['rep_pct'] + (1 - label_df['rep_incumbent']) * label_df['dem_pct']

label_df = label_df[['year', 'state', 'result', 'rep_incumbent', 'inc_pct']].set_index(['year', 'state'])

In [16]:
election_dfs = {
    'house': pd.read_csv('../processed_data/1976-2022-house-two-party-vote-by-state.csv'),
    'senate' : pd.read_csv('../processed_data/1976-2020-senate.csv'),
    'pres': pd.read_csv('../processed_data/1976-2020-president-two-party.csv')
}

feature_dfs = []

for i in range(2,25,2):
    for tp in election_dfs:
        temp_df = election_dfs[tp][['year','state', 'rep_pct']]
        temp_df.loc[:,'year'] = temp_df['year'] + i
        temp_df = temp_df.rename(columns = {'rep_pct': f'result_{tp}_{i}'})\
                        .set_index(['year','state'])
        feature_dfs.append(temp_df)

unemployment_df = pd.read_csv('../processed_data/1976-2023-unemployment.csv')

urbanicity_df = pd.read_csv('../processed_data/1970-2020-urbanicity.csv')\
                    .sort_values(by = 'year')\
                    .groupby('state')[['year', 'urbanicity']]\
                    .apply(lambda table: table.set_index('year')\
                                                .reindex(range(table['year'].min(), table['year'].max() + 1))\
                                                .reset_index()\
                                                .sort_values(by = 'year')\
                                                .ffill())\
                    .reset_index()\
                    .drop(columns = 'level_1')

demographic_dfs = {
    'race' : pd.read_csv('../processed_data/2010-2020-race-demographics.csv'),
    'education' : pd.read_csv('../processed_data/2006-2022-education.csv'),
    'urbanicity' : urbanicity_df,
    'unemployment' : unemployment_df[['year', 'state', 'unemployment_pct']],
    'religious' : pd.read_csv('../processed_data/2016-2020-religious.csv')
}

feature_dfs.extend([df.set_index(['year', 'state']) for df in demographic_dfs.values()])

econ_dfs = {
    'gdp' : pd.read_csv('../processed_data/1997-2022-gdp.csv'),
    'household_income' : pd.read_csv('../processed_data/1984-2022-median-household-income.csv'),
    'unemployment' : unemployment_df[['year', 'state', 'unemployment_pct_1', 'unemployment_pct_4']]
}

feature_dfs.extend([df.set_index(['year', 'state']) for df in econ_dfs.values()])

feature_df = pd.concat(feature_dfs, ignore_index = False, join = 'outer', axis = 1).reset_index()

stock_df = pd.read_csv('../processed_data/1971-2023-stocks.csv')

feature_df = pd.merge(feature_df, stock_df, on = 'year', how = 'left')\
                .dropna(axis = 1, how = 'all')

all_df = pd.merge(label_df, feature_df, on = ['year', 'state'], how = 'left')\
            .dropna(axis = 1, how = 'all')

In [17]:
codebook = {
    'all' : ['year', 'state', 'result', 'rep_incumbent', 'inc_pct', 'result_house_2',
       'result_senate_2', 'result_house_4', 'result_senate_4', 'result_pres_4',
       'result_house_6', 'result_senate_6', 'result_house_8',
       'result_senate_8', 'result_pres_8', 'result_house_10',
       'result_senate_10', 'result_house_12', 'result_senate_12',
       'result_pres_12', 'result_house_14', 'result_senate_14',
       'result_house_16', 'result_senate_16', 'result_pres_16',
       'result_house_18', 'result_senate_18', 'result_house_20',
       'result_senate_20', 'result_pres_20', 'result_house_22',
       'result_senate_22', 'result_house_24', 'result_senate_24',
       'result_pres_24', 'american_indian', 'asian', 'black', 'more_than_one',
       'pacific_islander', 'white', 'hispanic', 'white_non_hispanic',
       'hs_higher_pct', 'bachelor_higher_pct', 'urbanicity',
       'unemployment_pct', 'gdp_1', 'gdp_4', 'household_income_1',
       'household_income_4', 'unemployment_pct_1', 'unemployment_pct_4',
       'nasdaq_1', 'nasdaq_3', 'nasdaq_6', 'nasdaq_12', 'nasdaq_48', 'spy_1',
       'spy_3', 'spy_6', 'spy_12', 'spy_48'],
    
    'id' : ['year', 'state'],

    'labels' : ['result', 'rep_incumbent', 'inc_pct'],

    'all_features': ['result_house_2',
       'result_senate_2', 'result_house_4', 'result_senate_4', 'result_pres_4',
       'result_house_6', 'result_senate_6', 'result_house_8',
       'result_senate_8', 'result_pres_8', 'result_house_10',
       'result_senate_10', 'result_house_12', 'result_senate_12',
       'result_pres_12', 'result_house_14', 'result_senate_14',
       'result_house_16', 'result_senate_16', 'result_pres_16',
       'result_house_18', 'result_senate_18', 'result_house_20',
       'result_senate_20', 'result_pres_20', 'result_house_22',
       'result_senate_22', 'result_house_24', 'result_senate_24',
       'result_pres_24', 'american_indian', 'asian', 'black', 'more_than_one',
       'pacific_islander', 'white', 'hispanic', 'white_non_hispanic',
       'hs_higher_pct', 'bachelor_higher_pct', 'urbanicity',
       'unemployment_pct', 'gdp_1', 'gdp_4', 'household_income_1',
       'household_income_4', 'unemployment_pct_1', 'unemployment_pct_4',
       'nasdaq_1', 'nasdaq_3', 'nasdaq_6', 'nasdaq_12', 'nasdaq_48', 'spy_1',
       'spy_3', 'spy_6', 'spy_12', 'spy_48'
    ],

    'election_features' : [
        'result_house_2',
       'result_senate_2', 'result_house_4', 'result_senate_4', 'result_pres_4',
       'result_house_6', 'result_senate_6', 'result_house_8',
       'result_senate_8', 'result_pres_8', 'result_house_10',
       'result_senate_10', 'result_house_12', 'result_senate_12',
       'result_pres_12', 'result_house_14', 'result_senate_14',
       'result_house_16', 'result_senate_16', 'result_pres_16',
       'result_house_18', 'result_senate_18', 'result_house_20',
       'result_senate_20', 'result_pres_20', 'result_house_22',
       'result_senate_22', 'result_house_24', 'result_senate_24',
       'result_pres_24'
    ],

    'demographic_features' : [
        'american_indian', 'asian', 'black', 'more_than_one',
       'pacific_islander', 'white', 'hispanic', 'white_non_hispanic',
       'hs_higher_pct', 'bachelor_higher_pct', 'urbanicity',
       'unemployment_pct'
    ],

    'economic_features': [
        'gdp_1', 'gdp_4', 'household_income_1',
       'household_income_4', 'unemployment_pct_1', 'unemployment_pct_4',
       'nasdaq_1', 'nasdaq_3', 'nasdaq_6', 'nasdaq_12', 'nasdaq_48', 'spy_1',
       'spy_3', 'spy_6', 'spy_12', 'spy_48'
    ],

    'logit_columns' : ['inc_pct', 'result',
        'result_house_2',
       'result_senate_2', 'result_house_4', 'result_senate_4', 'result_pres_4',
       'result_house_6', 'result_senate_6', 'result_house_8',
       'result_senate_8', 'result_pres_8', 'result_house_10',
       'result_senate_10', 'result_house_12', 'result_senate_12',
       'result_pres_12', 'result_house_14', 'result_senate_14',
       'result_house_16', 'result_senate_16', 'result_pres_16',
       'result_house_18', 'result_senate_18', 'result_house_20',
       'result_senate_20', 'result_pres_20', 'result_house_22',
       'result_senate_22', 'result_house_24', 'result_senate_24',
       'result_pres_24',
       'american_indian', 'asian', 'black', 'more_than_one',
       'pacific_islander', 'white', 'hispanic', 'white_non_hispanic',
       'hs_higher_pct', 'bachelor_higher_pct', 'urbanicity', 'white_evangelist_pct',
       'unemployment_pct'
    ]
}

In [18]:
all_df.to_csv('../all_data.csv', index = False)

In [19]:
all_df_logit = all_df.copy()
thresh = 0.0001
for col in codebook['logit_columns']:
    all_df_logit.loc[:,col] = logit(all_df[col].apply(lambda x: max(x, thresh))\
                                   .apply(lambda x : min(x, 1-thresh)))

In [20]:
all_df_logit.to_csv('../all_data_logit.csv', index = False)